In [1]:
import os, json
from glob import glob
import numpy as np
from scipy import misc, ndimage
from scipy.ndimage.interpolation import zoom

from keras.utils.data_utils import get_file
from keras import backend as K
from keras.layers.normalization import BatchNormalization
from keras.utils.data_utils import get_file
from keras.models import Sequential
from keras.layers.core import Flatten, Dense, Dropout, Lambda
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.layers.pooling import GlobalAveragePooling2D
from keras.optimizers import SGD, RMSprop, Adam
from keras.preprocessing import image

Using Theano backend.
Using gpu device 0: GeForce GTX 1070 (CNMeM is disabled, cuDNN 5103)
/home/vijin/anaconda2/lib/python2.7/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)


In [3]:
def get_model(weights_path=None):
    model = Sequential()
    model.add(ZeroPadding2D((1,1),input_shape=(3,224,224)))
    model.add(Convolution2D(64, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(64, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(128, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(128, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(Flatten())
    model.add(Dense(4096, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(4096, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1000, activation='softmax'))

    if weights_path:
        model.load_weights(weights_path)

    return model

In [4]:
def get_batches(path, gen=image.ImageDataGenerator(), shuffle=True, batch_size=8, class_mode='categorical'):
    return gen.flow_from_directory(path, target_size=(224,224),class_mode=class_mode, shuffle=shuffle, batch_size=batch_size)

In [5]:
def finetune(model, batches):
    model.pop()
    for layer in model.layers: layer.trainable=False
    model.add(Dense(batches.nb_class, activation='softmax'))
    model.compile(optimizer=Adam(lr=0.001),
                loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [6]:
def fit(model, batches, val_batches, nb_epoch=1):
    model.fit_generator(batches, samples_per_epoch=batches.nb_sample, nb_epoch=nb_epoch,
                validation_data=val_batches, nb_val_samples=val_batches.nb_sample)

In [7]:
def test(model, path, batch_size=8):
        test_batches = get_batches(path, shuffle=False, batch_size=batch_size, class_mode=None)
        return test_batches, model.predict_generator(test_batches, test_batches.nb_sample)

In [8]:
vgg_19_model = get_model(weights_path='vgg19_weights.h5')

In [8]:
path = "/home/vijin/DeepLearning/catsVsDogs/"
batch_size = 64
batches = get_batches(path+'train', batch_size=batch_size)
val_batches = get_batches(path+'valid', batch_size=batch_size*2)

Found 24000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [9]:
finetune(vgg_19_model,batches)

In [10]:
fit(vgg_19_model,batches, val_batches, nb_epoch=1)
vgg_19_model.save_weights(path+'result/vgg_19_ft1.h5')

fit(vgg_19_model,batches, val_batches, nb_epoch=1)
vgg_19_model.save_weights(path+'result/vgg_19_ft2.h5')

fit(vgg_19_model,batches, val_batches, nb_epoch=1)
vgg_19_model.save_weights(path+'result/vgg_19_ft3.h5')

fit(vgg_19_model,batches, val_batches, nb_epoch=1)
vgg_19_model.save_weights(path+'result/vgg_19_ft4.h5')

Epoch 1/1
24000/24000 [==============================] - 455s - loss: 0.2090 - acc: 0.9459 - val_loss: 0.0960 - val_acc: 0.9690

In [14]:
#testing
vgg_19_model.load_weights(path+'result/vgg_19_ft4.h5')
test_batches, prediction = test(vgg_19_model,path+'test',batch_size=batch_size)

#writing to submission file
import re
files = test_batches.filenames
output_map = zip([re.match(r'.*?(\d+).*',x).group(1) for x in files] , [x[1] for x in prediction])
sub_file = open(path+'result/submission_1.csv', 'a')
sub_file.write('id,label\n')
sub_file.write('\n'.join('%s,%s' % x for x in output_map))
sub_file.close()

Found 12500 images belonging to 1 classes.
